In [1]:
# # # Konsumgruppe DF

import pandas as pd

df_produkter = pd.read_csv("C:/Users/Bruker/Tabell_03014_Detaljer.csv", encoding='ISO-8859-1',sep='\t')
df_produkter['konsumgruppe'] = df_produkter['konsumgruppe'].astype(str)
df_produkter = df_produkter.drop(columns=list(df_produkter.columns[2:39]))
df_produkter = df_produkter.drop(columns=["statistikkvariabel"])
df_produkter.rename(columns={"konsumgruppe":"Konsumgruppe"}, inplace=True)

df_produkter


,Konsumgruppe,2016,2017,2018,2019,2020,2021,2022
0,01.1.1 Brød og kornprodukter,2.8,-0.5,1.6,1.5,2.7,0.2,7.4
1,01.1.2 Kjøtt,2.5,-2.3,0.8,1.4,2.3,3.0,9.9
2,01.1.3 Fisk og sjømat,7.4,9.1,1.8,1.8,2.6,0.6,8.7
3,"01.1.4 Melk, ost og egg",1.3,0.6,3.1,0.8,2.5,-0.6,5.0
4,01.1.5 Oljer og fett,8.3,0.1,5.4,5.0,5.6,-0.2,12.2
...,...,...,...,...,...,...,...,...
88,12.4.0 Sosiale omsorgstjenester,0.5,1.9,4.2,2.2,2.2,1.1,0.0
89,12.5.2 Boligforsikring,.,.,.,.,.,.,.
90,12.5.4 Transportforsikring,.,.,.,.,.,.,.
91,12.6.2 Finansielle tjenester ellers,-0.5,2.6,-0.3,0.4,3.3,0.7,0.6


In [2]:
# # # Lønn DF

df_lønn = pd.read_csv("C:/Users/Bruker/Tabell_11419_Lønn.csv", encoding='ISO-8859-1',sep='\t')
df_lønn = df_lønn[(df_lønn["kjønn"] == "Begge kjønn") & (df_lønn["yrke"] == "Alle yrker") & (df_lønn["sektor"] == "Sum alle sektorer")]
df_lønn = df_lønn.drop(columns=["statistikkvariabel","statistikkmål","kjønn","yrke","sektor"] + [col for col in df_lønn.columns if "Heltidsansatte" in col or "Deltidsansatte" in col])
df_lønn.rename(columns=lambda x: x.replace(" I alt", ""), inplace=True)

df_lønn = df_lønn.replace(to_replace='.', value=0)

df_lønn.rename(columns={"næring (SN2007)":"Næring"}, inplace=True)

df_lønn


,Næring,2015,2016,2017,2018,2019,2020,2021,2022
0,03.2 Akvakultur,43570,45790,47760,48920,50020,51940,54240,57370
3,05_07_08_09.9 Bergverksdrift,41800,43000,44040,45620,47020,48290,50000,53240
6,06+09.1 Utvinning av råolje og naturgass inkl....,70070,71280,71700,74760,76490,76280,78320,84250
9,"10-12 Nærings-, drikkevare- og tobakksindustri",38890,40050,41100,42240,43780,44330,46300,48220
12,"13-15 Tekstil-, bekledning- og lærvareindustri",36560,38900,38550,40060,41450,42990,45480,47090
...,...,...,...,...,...,...,...,...,...
240,94.9 Aktiviteter i andre medlemsorganisasjoner,40510,41810,42820,43810,45510,46450,48060,49360
243,95 Reparasjon av varer til personlig bruk,35730,36390,36870,37900,38950,40400,41160,42690
246,96 Annen personlig tjenesteyting,30560,32090,32500,33350,34150,35250,37130,38770
249,97 Lønnet arbeid i private husholdninger,0,36290,36760,38490,39380,41450,42700,44270


In [46]:
# # # Plot Data

import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

konsumgruppe_label = widgets.Label('Select Konsumgruppe:')
options_konsumgruppe = ['Konsumgruppe'] + df_produkter['Konsumgruppe'].unique().tolist()
dropdown_konsumgruppe = widgets.Dropdown(options=options_konsumgruppe)
display(widgets.VBox([konsumgruppe_label, dropdown_konsumgruppe]))


naering_label = widgets.Label('Select Næring:')
options_naering = ['Næring'] + df_lønn['Næring'].unique().tolist()
dropdown_naering = widgets.Dropdown(options=options_naering)
display(widgets.VBox([naering_label, dropdown_naering]))


plot_button = widgets.Button(description='Plot Combined Graph', visible=False)
display(plot_button)


def plot_data(data):
    data.iloc[:, 1:] = data.iloc[:, 1:].astype(float)
    plt.plot(data.columns[1:], data.iloc[0, 1:].astype(float))

def update_plot_konsumgruppe(change):
    global Konsumgruppe_df
    Konsumgruppe_df = df_produkter[df_produkter['Konsumgruppe'] == change.new]
    plot_data(Konsumgruppe_df)

def update_plot_naering(change):
    global Næring_df
    Irke_df = df_lønn[df_lønn['Næring'] == change.new]
    Irke_df.iloc[:, 1:] = Irke_df.iloc[:, 1:].astype(float)
    Næring_df = Irke_df.iloc[:, 1:].pct_change(axis=1) * 100
    Næring_df = pd.concat([Irke_df.iloc[:, 0], Næring_df], axis=1)
    plot_data(Næring_df)


def plot_combined(button):
    plot_data_combined(Konsumgruppe_df, Næring_df)

def plot_data_combined(df1, df2):
    plt.figure(figsize=(12, 8))
    plt.plot(df1.columns[1:], df1.iloc[0, 1:].astype(float))
    plt.plot(df2.columns[1:], df2.iloc[0, 1:].astype(float))
    plt.xlabel("Year")
    plt.ylabel("Percentage Variation")
    plt.title("Percentage variation by Year")
    plt.show()

dropdown_konsumgruppe.observe(update_plot_konsumgruppe, names='value')
dropdown_naering.observe(update_plot_naering, names='value')
plot_button.on_click(plot_combined)


Button(description='Plot Combined Graph', style=ButtonStyle())